## Sparkをはじめてみよう
Sparkを使って、README.mdを読み込み、いろいろと操作してみます。

In [ ]:
textFile = sc.textFile("../README.md")

In [ ]:
# 行数をカウント
textFile.count()

In [ ]:
# "Spark"を含む行を抽出。
sparkLines = textFile.filter(lambda l: "Spark" in l)
sparkLines.count()

### filter-> pythonのfilterと同じ

In [ ]:
# "Spark"を含む行を全件取得
sparkLines.collect()

In [ ]:
# 出現単語ごとの件数を取得
textFile.flatMap(lambda l: l.strip().split()).map(lambda w: (w, 1)).reduceByKey(lambda x, y: x + y).collect()

### flatMap -> mapしつつ、内部のコレクションを展開する関数。[[1,2], [3,4], [4,5]]をflatMapすると、[1,2,3,4,5]になる。
### map -> pythonのmapと同じ
### reduceByKey -> (key, value)のコレクションに対して、キーごとに値を集計する。

## RDDの作り方
RDDの作り方は、大きく分けて２通りあります。

In [ ]:
# コレクションオブジェクトから作成する
l = [1, 2, 3]
rdd = sc.parallelize(l)
rdd.collect()

In [ ]:
# 外部から読み込む
rdd = sc.textFile("../README.md")
rdd.count()

## RDDの処理イメージ
RDDのTransformationとActionがどのように実行されるのか見てみます。

In [ ]:
# テキストファイルからRDD作成。この時点ではRDDオブジェクトが作成されるだけで、実際の読み込みは発生しない。
linesRDD = sc.textFile("../README.md")
print type(linesRDD)

In [ ]:
# "Spark"というワードを持つ行のみ抽出。ここでも新たなRDDオブジェクトが作成されるだけ。
filteredRDD =  linesRDD.filter(lambda l: "Spark" in l)
print type(filteredRDD)

In [ ]:
# 行数取得。ここでやっとテキストファイルから読み込んでフィルタするという操作が実行される。
filteredRDD.count()

## RDDのキャッシュ

In [ ]:
textFile = sc.textFile("../README.md")

In [ ]:
# cache()を呼び出すことで、メモリにキャッシュすることが指示されます。ここではまだキャッシュされません。
cached = textFile.filter(lambda l: "Spark" in l).cache()

In [ ]:
# ここで、テキストファイルからの読み込みとフィルタ処理が行われた上で、結果がメモリにキャッシュされます。
cached.count()

In [ ]:
# メモリからfilter処理までが終わった状態のデータを読み込み、処理します。テキストファイルが再び読み込まれることはありません。
import datetime
cached.saveAsTextFile("README-filtered-{}.md".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S")))

## Sparkの管理画面で実行を確認する

### Shuffleのないケース

In [ ]:
sc.textFile("../licenses/*.txt").filter(lambda l: "Copyright" in l).first()

[管理画面で確認](http://localhost:4040/)

### Shuffleのあるケース

In [ ]:
sc.textFile("../licenses/*.txt").flatMap(lambda l: l.split()).map(lambda w: (w, 1)).reduceByKey(lambda x, y: x + y).take(1)

[管理画面で確認](http://localhost:4040/)

## DataFrameを動かしてみよう

In [ ]:
# DataFrameを使うためのエントリーポイントとなるオブジェクト作成
from pyspark.sql import SparkSession
spark =  SparkSession.builder.appName("pycon2016").getOrCreate()

In [ ]:
# 下記のようなjsonがあります。
%cat ../examples/src/main/resources/people.json

In [ ]:
# これを読み込みます。
df = spark.read.json("../examples/src/main/resources/people.json")
df.show()

In [ ]:
# 特定のカラムを選択します。
df.select("name").show()

In [ ]:
# 絞り込み選択
df.filter(df['age'] > 21).show()

In [ ]:
# 集約関数
df.groupBy("age").count().show()

In [ ]:
# メモリ上にテンポラリテーブルとして保存することもできます。
df.createOrReplaceTempView("people")

In [ ]:
# テーブルには直接SQLを発行することができます。戻り値はDataFrameです。
res = spark.sql("select age from people where name = 'Andy'")
res.show()

In [ ]:
# なお、スキーマについてはデータから類数されています。
df.printSchema()

In [ ]:
# もちろん、データを書き込むことも可能です。jsonで読んだデータをcsvで書き込んでみます。
df = spark.read.json("../examples/src/main/resources/people.json")
df.select(df["name"], df["age"] + 10).write.mode("overwrite").csv("/tmp/new_people.csv")
spark.read.csv("/tmp/new_people.csv").show()

## 様々なデータソース

In [ ]:
# csv
dfCsv = spark.read.csv("../examples/src/main/resources/test.csv")
dfCsv.show()

In [ ]:
# parquet
dfParquet = spark.read.parquet("../examples/src/main/resources/users.parquet")
dfParquet.show()

## DataFrameを使うメリット

In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as func

# 元データ
raw = [("Pete", "dev", 20), ("Keith", "dev", 22),  ("Roger", "sales", 30), ("John", "sales", 28)]
rdd = sc.parallelize(raw)
df = spark.createDataFrame(sc.parallelize(raw).map(lambda r: Row(name=r[0], department=r[1], age=r[2])))

In [ ]:
# RDDで処理するケース
rdd.map(lambda r: (r[1], (float(r[2]), 1))).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda v: v[0] / v[1]).collect()

In [ ]:
# DataFrameで処理するケース
df.groupBy("department").agg(func.avg("age")).show()